In [1]:
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k
import scipy
from scipy.sparse import coo_matrix, eye, hstack, lil_matrix
from skopt import forest_minimize
from pycarol import Carol, Staging, ApiKeyAuth, Storage, PwdAuth
from pycarol.data_models import DataModel
import pandas as pd
import datetime
import json
import re
import time
import math
import seaborn as sns
import matplotlib.pyplot as plt
import pickle 

/Users/felipe/opt/miniconda3/envs/recommendation/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
login_dont = Carol(domain='monitoriaqa',
app_name=' ',
organization='totvs',
auth=PwdAuth('fmendes@totvs.com.br', 'Intxdx70248821*'))

In [3]:
df = Staging(login_dont).fetch_parquet(staging_name='articles',
                        connector_name='carol_connect_zerado',
                        cds=True
                        )

100%|██████████| 60/60 [02:01<00:00,  2.02s/it]


In [5]:
categories = Staging(login_dont).fetch_parquet(staging_name='categories',
                        connector_name='carol_connect_zerado',
                        cds=True
                        )
categories = categories.astype(str)

100%|██████████| 5/5 [00:07<00:00,  1.51s/it]


In [7]:
sections = Staging(login_dont).fetch_parquet(staging_name='sections',
                        connector_name='carol_connect_zerado',
                        cds=True
                        )
sections = categories.astype(str)

100%|██████████| 26/26 [00:19<00:00,  1.30it/s]


In [8]:
categories.iloc[124].html_url

'https://atendimento-homolog.totvs.com/hc/pt-br/categories/203876667-RM-Categoria'

In [22]:
categories.name.values

array([' Atualizações Totvs', 'Ambiente', 'Analytics', 'Aprendizado',
       'Auto atendimento', 'Bemacash', 'Bemacash FLY01 Vestuário',
       'CLOUD', 'CMNET', 'CMNET', 'CMNet Reservas e Higs', 'CST', 'CST',
       'CST Adm Gente', 'CST Adm Gente', 'Cadastros', 'Categoria',
       'Categoria de teste - CST', 'Central de Oportunidades  ',
       'Centro de Serviços', 'Ciclo de Vida', 'Ciclo de Vida', 'Cloud',
       'Cloud', 'Consumer', 'Copiloto', 'CoreBanking', 'CoreBanking',
       'DON - CHRONUS', 'Datasul', 'Datasul', 'Datasul', 'Datasul',
       'Documentações', 'Documentações', 'Documentos', 'ESocial', 'Eleve',
       'Enterprise Service Bus', 'Financeiro', 'Financial Services',
       'Financial Services', 'Financial Services', 'Fiscal Varejo',
       'Fiscal Varejo', 'Fluig', 'Fluig', 'Fluig', 'Fluig', 'Fly01',
       'Fly01', 'Fly01', 'Gemco', 'Gemco Anywhere', 'Gemco Anywhere',
       'Geral', 'Geral', 'Geral', 'Geral', 'Geral',
       'Governança de Serviços', 'Healthcare'

In [19]:
categories[categories.name == 'WinThor']['id']

125    360001488852.0
127    360001488832.0
Name: id, dtype: object

In [23]:
categories[categories.id == '360003678772.0']

,__v,brand_id,brand_name,created_at,description,html_url,id,idinternalmdb,locale,name,outdated,position,source_locale,updated_at,url


In [4]:
df = df.astype(str)

In [5]:
df.shape

(36332, 26)

In [6]:
df[(df.source_locale == 'es')].shape

(2128, 26)

In [10]:
df.iloc[0].html_url

'https://centraldeatendimento.fluig.com/hc/pt-br/articles/360034984293-FL-ANA-Como-configurar-widget-Analytics-no-fluig'

In [8]:
df[df.draft == 'True'].shape

(2206, 26)

In [14]:
df.iloc[9562]

__v                                                                  0
author_id                                               378119603651.0
body                 <p><strong>Dúvida<br></strong>Como utilizar a ...
brand_id                                                       1509248
brand_name                                                       TOTVS
comments_disabled                                                False
created_at                                    2019-11-05T14:17:23.000Z
draft                                                             True
edited_at                                     2019-11-05T14:17:23.000Z
html_url             https://centraldeatendimento.totvs.com/hc/pt-b...
id                                                      360037893454.0
idinternalmdb                                 5ea30436669e459227dd569c
label_names          ["brasil","mercadoria","winthor","versao 29","...
name                 WINT - Como utilizar a rotina 1143 - Sobra de ...
outdat

In [37]:
sections.columns

Index(['__v', 'brand_id', 'brand_name', 'created_at', 'description',
       'html_url', 'id', 'idinternalmdb', 'locale', 'name', 'outdated',
       'position', 'source_locale', 'updated_at', 'url'],
      dtype='object')

In [25]:
sections.iloc[15].id

'115001274268.0'

In [27]:
sections.iloc[15]

__v                                                              0
brand_id                                                   7539207
brand_name                                             CST Sandbox
created_at                                2017-05-24T14:46:48.000Z
description                                                       
html_url         https://cstsandbox.zendesk.com/hc/pt-br/catego...
id                                                  115001274268.0
idinternalmdb                             5b6980246c84568045bd6151
locale                                                       pt-br
name                                                         Geral
outdated                                                     False
position                                                         0
source_locale                                                pt-br
updated_at                                2018-05-10T12:16:35.000Z
url              https://cstsandbox.zendesk.com/api/v2/help_ce

In [30]:
df = df.astype(str)

In [29]:
sections[sections.id.str.contains('54308')]

,__v,brand_id,brand_name,created_at,description,html_url,id,idinternalmdb,locale,name,outdated,position,source_locale,updated_at,url


In [14]:
df.iloc[9562].html_url

'https://centraldeatendimento.totvs.com/hc/pt-br/articles/360035446493-LG-FAT-Verificar-duplicatas-pagas-ou-emitidas-no-cancelamento-da-nota-fiscal'

In [35]:
df[df.section_id.str.contains('1150012')]['section_id']

367      115001251127.0
490      115001251127.0
534      115001251127.0
549      115001251127.0
566      115001251127.0
573      115001251127.0
778      115001251127.0
1236     115001251127.0
1351     115001251127.0
1851     115001251127.0
2177     115001251127.0
2446     115001251127.0
2583     115001251127.0
2605     115001251127.0
2713     115001251127.0
2714     115001251127.0
2719     115001251127.0
2721     115001251127.0
2768     115001251127.0
2783     115001251127.0
2874     115001251127.0
2933     115001251127.0
2938     115001251127.0
3005     115001251127.0
3121     115001251127.0
3153     115001251127.0
3260     115001251127.0
3277     115001251127.0
3323     115001251127.0
3788     115001251127.0
              ...      
35658    115001251127.0
35820    115001251127.0
35881    115001251127.0
35906    115001251127.0
36191    115001251127.0
36252    115001251127.0
36379    115001251127.0
36438    115001251127.0
36483    115001251127.0
36568    115001251127.0
36673    1150012

In [17]:
df.iloc[952].section_id

206933207.0

In [11]:
sections[sections.id == '206954428.0']

,__v,brand_id,brand_name,created_at,description,html_url,id,idinternalmdb,locale,name,outdated,position,source_locale,updated_at,url


In [12]:
df[df.id.str.contains('360037893454')]['id']

9562    360037893454.0
Name: id, dtype: object

In [7]:
sample = df[(df.source_locale == 'pt-br') & (~df.body.isnull())]

In [8]:
sample.shape

(34187, 26)

In [9]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    return " ".join(re.sub(r'\s([?.!"](?:\s|$))', r'\1', text).strip().split())

In [10]:
def get_question(body):
    
    body = body.replace('\n', '')

    m = re.search('(?<=(Dúvida)|(Duvida)).*?(?=Ambiente)', body, re.IGNORECASE)
    if m:
        return remove_html_tags(m.group(0))

    m = re.search('(?<=(Ocorrência)|(Ocorrencia)).*?(?=Ambiente)', body, re.IGNORECASE)
    if m:
        return remove_html_tags(m.group(0))
    
    return np.nan

In [11]:
def get_question_type(body):
    
    body = body.replace('\n', '')

    m = re.search('(?<=(Dúvida)|(Duvida)).*?(?=Ambiente)', body, re.IGNORECASE)
    if m:
        return 'question'

    m = re.search('(?<=(Ocorrência)|(Ocorrencia)).*?(?=Ambiente)', body, re.IGNORECASE)
    if m:
        return 'occurrence'
    
    return np.nan

In [12]:
def get_environment(body):
    
    body = body.replace('\n', '')

    m = re.search('(?<=Ambiente).*?(?=Solução)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return remove_html_tags(m.group(0))

In [13]:
def get_solution(body):
    
    body = body.replace('\n', '')

    m = re.search('(?<=Solução)(?s)(.*$)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return m.group(0)

In [14]:
df['question'] = df.body.apply(get_question)
df['question_type'] = df.body.apply(get_question_type)
df['environment'] = df.body.apply(get_environment)
df['solution'] = df.body.apply(get_solution)

<ipython-input-13-d7c59c3a7d97>:5: DeprecationWarning: Flags not at the start of the expression '(?<=Solução)(?s)(.*$' (truncated)
  m = re.search('(?<=Solução)(?s)(.*$)', body, re.IGNORECASE)


In [15]:
empty = sample[(sample['question'].isnull()) | (sample['question_type'].isnull()) | (sample['environment'].isnull()) | (sample['solution'].isnull())]

In [16]:
df = df.dropna(subset=['question', 'question_type', 'environment', 'solution'])

In [17]:
sample.shape

(33557, 30)

In [18]:
empty.shape

(630, 30)

In [11]:
df.iloc[0]

__v                                                                  0
author_id                                                  1.96601e+10
body                 <p><br><strong>Dúvida</strong><strong><br></st...
brand_id                                                       2911068
brand_name                                                       Fluig
comments_disabled                                                False
created_at                                    2019-08-27T16:20:33.000Z
draft                                                            False
edited_at                                     2019-08-27T16:20:37.000Z
html_url             https://centraldeatendimento.fluig.com/hc/pt-b...
id                                                         3.60035e+11
idinternalmdb                                 5ea3019c669e459227dc4d93
label_names          ["fluig_analytics","fluig_1.6.5","fluig_analyt...
name                 FL - ANA - Como configurar widget Analytics no...
outdat

In [19]:
empty.head()

,__v,author_id,body,brand_id,brand_name,comments_disabled,created_at,draft,edited_at,html_url,...,title,updated,updated_at,url,vote_count,vote_sum,question,question_type,environment,solution
43,0,1.810634e+10,<p><strong>Dúvida</strong><strong><br></strong...,2911068,Fluig,False,2019-08-30T17:36:43.000Z,False,2019-09-03T16:57:40.000Z,https://centraldeatendimento.fluig.com/hc/pt-b...,...,FL - BPM - Como configurar corretamente o widg...,False,2019-09-03T16:57:40.000Z,https://fluigexterno.zendesk.com/api/v2/help_c...,0,0,Como configurar corretamente o widget de Gráfi...,question,NaN,NaN
54,0,3.657333e+11,<p><strong>Tema </strong><strong><br></strong>...,2911068,Fluig,False,2019-07-31T16:14:16.000Z,False,2019-08-02T14:08:12.000Z,https://centraldeatendimento.fluig.com/hc/pt-b...,...,FL - ECM - Principais dúvidas sobre permissão ...,False,2019-08-09T18:57:05.000Z,https://fluigexterno.zendesk.com/api/v2/help_c...,1,1,s sobre permissão e segurança das pastas e dos...,question,NaN,NaN
73,0,2.502262e+10,<p><strong><u>Serviço</u></strong></p>\n<p><u>...,2911068,Fluig,False,2020-03-03T17:54:14.000Z,False,2020-03-03T17:54:19.000Z,https://centraldeatendimento.fluig.com/hc/pt-b...,...,FL - BPM - Utilização de Webservices - startPr...,False,2020-03-03T17:54:19.000Z,https://fluigexterno.zendesk.com/api/v2/help_c...,0,0,NaN,NaN,NaN,NaN
76,0,1.810634e+10,"<p><strong>Método:<br></strong>getCardValue (""...",2911068,Fluig,False,2020-02-28T17:47:52.000Z,False,2020-03-03T17:44:02.000Z,https://centraldeatendimento.fluig.com/hc/pt-b...,...,FL - BPM - hAPI - getCardValue,False,2020-03-03T17:44:02.000Z,https://fluigexterno.zendesk.com/api/v2/help_c...,0,0,NaN,NaN,NaN,NaN
81,0,2.502262e+10,<p><strong><u>Método</u></strong></p>\n<p>setC...,2911068,Fluig,False,2020-02-28T18:22:42.000Z,False,2020-02-28T18:25:58.000Z,https://centraldeatendimento.fluig.com/hc/pt-b...,...,FL - BPM - hAPI - setCardValue,False,2020-02-28T18:31:53.000Z,https://fluigexterno.zendesk.com/api/v2/help_c...,0,0,NaN,NaN,NaN,NaN


In [20]:
empty.to_excel('articles_with_missing_information.xlsx', index=False)

In [22]:
questions = sample.question.values

In [25]:
sample.iloc[0].question == questions[0]

True

In [23]:
len(questions)

33557

In [24]:
len(set(questions))

33142

In [26]:
with open('questions', "bw") as f:
        pickle.dump(questions, f)

In [38]:
dupl = df[df.duplicated(['question'])]

KeyError: Index(['question'], dtype='object')